# QA Model Runs notebook

Notebook for pre-release QA process, running 2 scenarios from a previous model version and comparing results.

⚠️ TODO: Set the paths to the two results files from Azure that you want to run, in the variable "results_dict", together with the scheme code. Example given in the first cell.

This notebook:

1. Downloads results files from Azure Results container
2. Converts the params in the results files into model params .json files, makes minor edits, starts containers with the modified json files on dev
3. Checks the status of the runs
4. When the runs are completed, compares the dev model run results to the results downloaded from Azure.

⚠️ Note that this notebook will only work if there have not been breaking changes in the params files between the model versions being tested. If there have been breaking changes, you will need to add these into the cell where the parameters are edited.

The notebook produces and displays dataframes comparing results from the previous model version with the dev version of the model. You will have to use your own eyes 👀 to check for differences and record any findings/observations on the relevant Prerelease QA checklist GitHub issue. 

In [ ]:
# ⚠️ Select appropriate scenarios from previous model version
# You may have to run one with params-sample from previous model version if this does not already exist

results_dict = {
    # 1 "real" scenario from previous model version
    "RX1": {
        "results_path": "prod/vX.X/RX1/scenarioname.json.gz"
    },
    # 1 scenario using sample params from previous model version
    "RX2": {
        "results_path": "prod/vX.X/RX2/scenarioname.json.gz"
    }
}

In [ ]:
# Get params from Azure
%cd ../..

import os
import json
import pandas as pd
from dotenv import load_dotenv
from nhpy import az, process_params, process_results

%load_ext autoreload
%autoreload 2

load_dotenv()
account_url = os.getenv("AZ_STORAGE_EP")
results_container = os.getenv("AZ_STORAGE_RESULTS")

## Get parameters from Azure

In [ ]:
# Get scenarios that have been run, where results are stored on Azure


results_connection = az.connect_to_container(account_url, results_container)


for trust in results_dict.keys():

    results_path = results_dict[trust]["results_path"]
    results_json = az.load_results_gzip_file(results_connection, results_path)
    results_dict[trust]["results_old"] = results_json

In [ ]:
# Get params only from results JSONs, edit scenario name, save to queue folder
# ⚠️ WARNING!!! If there have been any breaking changes to params you will have to implement them here

if not os.path.exists("queue"):
    os.makedirs("queue")

filenames = []
for trust in results_dict.keys():
    params = results_dict[trust]["results_old"]["params"].copy()
    params["scenario"] = params["scenario"] + "-testdev"
    params_filename = f"{params['dataset'] + '-' + params['scenario']}.json"
    params["app_version"] = "dev"
    params["user"] = "ds-team"
    params["viewable"] = False
    # ⚠️  FOR v4.2 ONLY: REMOVE LINE BELOW IN NEXT MODEL VERSION TESTING
    params["inequalities"] = {"level_up": [], "zero_sum": [],"level_down": []}
    with open(os.path.join("queue", params_filename), "w") as f:
        json.dump(params, f)
    results_dict[trust]["new_params"] = params
    filenames.append(params_filename)

## Start new model runs on dev using nhp_aci

In [ ]:
from nhp.aci.run_model import create_model_run
from nhp.aci.run_model.helpers import validate_params
from nhpy.run_full_results import _track_container_status
from nhpy.utils import _construct_results_path

responses = []
for f in filenames:
    with open(os.path.join("queue", f), "rb") as fopen:
        params = json.load(fopen)
        validate_params(params, params["app_version"])
        metadata = create_model_run(params, params["app_version"])
        # Use container creation datetime
        params["original_datetime"] = params["create_datetime"]
        params["create_datetime"] = (
            metadata["create_datetime"] if metadata else ""
            )
        responses.append(metadata)
        results_dict[params["dataset"]]["new_results_path"] = _construct_results_path(params)["json_path"]


In [ ]:
# ⚠️ ⚠️ ⚠️  Need to refactor this to automatically poll every 120 seconds show info in the same way as run_full_results
# ⚠️ ⚠️ ⚠️  in the meantime just run this cell every now and then till you see your runs have completed

from pprint import pprint
from nhp.aci.status.list_current_model_runs import get_current_model_runs

current_runs = get_current_model_runs()

pprint(current_runs)

## Wait for runs to be completed ⌚

This normally takes about 15 mins. Your runs should say "detail_status": "Completed" (and then disappear from the list)

## Use completed dev run results

In [ ]:
# Read new model runs from Azure and store in the results_dict

for trust in results_dict.keys():
    results_path = results_dict[trust]["new_results_path"]
    results_json = az.load_results_gzip_file(results_connection, results_path)
    results_dict[trust]["results_new"] = results_json
    print(results_path)

In [ ]:
# Compare and save to CSV
from datetime import date

trusts = list(results_dict)
df_list = [process_results.compare_results(results_dict, t) for t in trusts]
(

    pd.concat(df_list)
    .reset_index()
    .groupby(["trust", "pod", "measure"])
    .sum()
    .to_csv(f"QA_default_results_{date.today()}.csv")
)

In [ ]:
# Compare and save to CSV



sc_list = [process_results.compare_stepcounts(results_dict, t) for t in trusts]
(
    pd.concat(sc_list)
    .reset_index()
    .fillna("-")
    .groupby(["trust", "change_factor", "measure", "strategy"])
    .sum(numeric_only=True)
    .to_csv(f"QA_stepcounts_{date.today()}.csv")
)